<a href="https://colab.research.google.com/github/evinracher/3008410-intelligent-systems/blob/main/week2/exercise1/Fine_Tunning_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%env TF_USE_LEGACY_KERAS=1

!pip -q uninstall -y keras transformers tensorflow tf-keras safetensors
!pip -q install \
  "tensorflow==2.20.0" \
  "tf-keras" \
  "transformers==4.49.0" \
  "safetensors==0.4.5" \
  "datasets==2.20.0" \
  "evaluate==0.4.2" \
  "accelerate==0.34.2" \
  "sentencepiece==0.2.0" rouge_score nltk

env: TF_USE_LEGACY_KERAS=1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, TFAutoModelForSeq2SeqLM, create_optimizer, AdamWeightDecay, pipeline
from datasets import load_dataset
import tensorflow as tf
from datasets import Dataset
import evaluate
import numpy as np
import torch
import os

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
folder_path = r"C:\Users\John\Downloads\Universidad\\AI Generativa Course\Exercises\Exercise2"
dataset_name = "eng_small.csv"
path = os.path.join(folder_path, dataset_name)
print(path)
data = Dataset.from_csv(path, encoding='utf-8')
data = data.train_test_split(test_size=0.1)
print(data)

C:\Users\John\Downloads\Universidad\\AI Generativa Course\Exercises\Exercise2\eng_small.csv
DatasetDict({
    train: Dataset({
        features: ['engl', 'spa'],
        num_rows: 14074
    })
    test: Dataset({
        features: ['engl', 'spa'],
        num_rows: 6033
    })
})


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")  #google/flan-t5-small
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small") #google/flan-t5-small
#model = export_and_get_onnx_model('t5-small')

prefix = "translate: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True) #max length was 128
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
tokenized_data = data.map(preprocess_function, batched=True, remove_columns=["engl", "spa"])

Map:   0%|          | 0/14074 [00:00<?, ? examples/s]

Map:   0%|          | 0/6033 [00:00<?, ? examples/s]

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")
optimizer = AdamWeightDecay(learning_rate=2e-4, weight_decay_rate=0.01) #2e-5 was before wd was 1e-2, Typically, 1e-4 and 3e-4 work well for most problems

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
tf_test_set = model.prepare_tf_dataset(
    tokenized_data["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
epochs = 5
model.compile(optimizer=optimizer)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=epochs, callbacks=None)

Epoch 1/5

879/879 [==============================] - 770s 845ms/step - loss: 2.2588 - val_loss: 1.5672
Epoch 2/5
879/879 [==============================] - 721s 820ms/step - loss: 1.6173 - val_loss: 1.2618
Epoch 3/5
879/879 [==============================] - 700s 796ms/step - loss: 1.3303 - val_loss: 1.1035
Epoch 4/5
879/879 [==============================] - 711s 809ms/step - loss: 1.1491 - val_loss: 1.0097
Epoch 5/5
879/879 [==============================] - 797s 906ms/step - loss: 1.0180 - val_loss: 0.9469


In [ ]:
# Guarda el modelo entrenado
folder_path = 'model'
model_name = "NMT-epocs-" + str(epochs)
path = os.path.join(folder_path, model_name + ".h5")
model.save_pretrained(path)
del model

In [ ]:
#Para inferir desde aquí.
model_name = "NMT-epocs-" + str(epochs)
path = os.path.join(folder_path, model_name + ".h5")

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = TFAutoModelForSeq2SeqLM.from_pretrained(path, pad_token_id=tokenizer.eos_token_id)

summarizer = pipeline("translation",
    model=model,
    tokenizer=tokenizer,
    framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at model\NMT-epocs-5.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
C:\Users\John\miniconda312\Lib\site-packages\transformers\pipelines\__init__.py:1156: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(
Device set to use 0


In [ ]:
import timeit
start_time = timeit.default_timer()

text = "translate: it's summer it is nice to go to the beach"
print(summarizer(text, min_length=4, max_length=100))

elapsed = timeit.default_timer() - start_time
print(f"time: {round(elapsed,2)} seconds")

[{'translation_text': 'Es buen momento de ir a la boca.'}]
time: 3.91 seconds


# Exercise

- This time use the larger dataset (eng.csv), use the same sentence and see the results.
- Modify the code to graph and report the Rouge metric (*)
  

If you want to try other tokenizers, see next

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True) #max length was 128
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
from transformers import AlbertTokenizer, AlbertModel

tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
model = AlbertModel.from_pretrained("albert-base-v2")

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True) #max length was 128
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
from transformers import RobertaTokenizer, RobertaModel

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True) #max length was 128
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
from transformers import ElectraTokenizer, ElectraModel

tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
mymodel = ElectraModel.from_pretrained("google/electra-small-discriminator")

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["engl"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["spa"], max_length=128, truncation=True) #max length was 128
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

### Exercise

- Change summarize by translate
- Graph the Rouge metric
